# CNNs for Image Classifocation with Pytorch

In [ ]:
! pip install --quiet "setuptools==59.5.0" "ipython[notebook]" "torchmetrics>=0.7" "torchvision" "seaborn" "torchviz"
! rm -rf ./logs/

In [ ]:
from IPython.display import Image
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose
from torchmetrics.classification import MulticlassAccuracy
from torchviz import make_dot
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

from google.colab import drive

%load_ext tensorboard

drive.mount('/content/drive/')

writer = SummaryWriter()

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

# I. First step: Simple Convolutional Neural Network




### Collecting and loading the data

In [ ]:
# Download training data from open datasets
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform = Compose([
    ToTensor()])
)

# Download test data from open datasets
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=Compose([
    ToTensor()])
)

# Display information about the data
print( 'Train data: ', training_data)
print( 'Test data: ', test_data)

In [ ]:
# Split training data into test and validation set
train_dataset, val_dataset = torch.utils.data.random_split(training_data, [0.8,0.2])

# Create data loaders
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Check our data's dimensions
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.size()}")
    print(f"Shape of y: {y.size()}")
    break

### Exploring the data

In [ ]:
# Display a few datapoints
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Building and training the model

### Architecture

In [ ]:
# Our class extends the nn.Module class
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        # Feature extraction stack
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1,6,(5,5),padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d((2,2),stride=2),
            nn.Conv2d(6,16,(5,5)),
            nn.Sigmoid(),
            nn.AvgPool2d((2,2),stride=2),
        )

        # Classification stack
        self.dense_stack = nn.Sequential(
            nn.LazyLinear(120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.flatten(x)
        logits = self.dense_stack(x)

        return logits

model = LeNet()


print(model)

In [ ]:
# Let us plot the model's graph and each layer's shapes
# The torchviz library requires us to pass a dummy batch to the model's forward()
batch = next(iter(train_dataloader))
yhat = model(batch[0])

make_dot(yhat, params=dict(list(model.named_parameters()))).render("model_torchviz", format="png")

In [ ]:
# Display information about our model's layers
summary(model.to(device), (1,28,28))

### Training

In [ ]:
# Create the training function
def train(dataloader, model, metric, loss_fn, optimizer, epoch, device):
    size = len(dataloader.dataset)
    outputs_list = []

    # Iterate over all batches. pbar:= allows us to display our training's progression with a progress bar.
    for batch, (X, y) in (pbar:= tqdm(enumerate(dataloader), total=len(dataloader))):
        # Need to convert X and y dtype to implement VGG16 later
        X = X.to(device).float()
        y = y.to(device).long()

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Add batch lostt to tensorboard's writer
        writer.add_scalar("Loss/train", loss, batch)

        # Collect batch metrics in a list
        outputs_list.append({"loss": loss, "preds": pred, "labels": y})

        # Update progress bar
        pbar.set_description(desc=f"Train - Loss:{loss:.2f}")

    # Join batch metrics and compute accuracy
    preds = torch.cat([x["preds"] for x in outputs_list]).detach().cpu()
    labels = torch.cat([x["labels"] for x in outputs_list]).detach().cpu()
    loss = torch.stack([x["loss"] for x in outputs_list]).mean()
    acc = metric(preds, labels)
    
    # Log everything to tensorboard's writer
    writer.add_scalar(f"train/loss", loss, epoch)
    writer.add_scalar(f"train/acc", acc, epoch)

    print(f"Train - Loss: {loss:.2f}, Acc: {acc}")

In [ ]:
def test(dataloader, model, metric, loss_fn, epoch, device):
    outputs_list = []
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    # We freeze the gradients at evaluation
    with torch.no_grad():
        for batch, (X, y) in (pbar:= tqdm(enumerate(dataloader), total=len(dataloader))):
            X = X.to(device).float()
            y = y.to(device).long()
            pred = model(X)
            test_loss = loss_fn(pred, y)
            outputs_list.append({"loss": test_loss, "preds": pred, "labels": y})

            pbar.set_description(desc=f"Val - Loss:{test_loss:.2f}")

    preds = torch.cat([x["preds"] for x in outputs_list]).detach().cpu()
    labels = torch.cat([x["labels"] for x in outputs_list]).detach().cpu()
    loss = torch.stack([x["loss"] for x in outputs_list]).mean()
    acc = metric(preds, labels)
    
    writer.add_scalar(f"val/loss", loss, epoch)
    writer.add_scalar(f"val/acc", acc, epoch)    

    print(f"Val - Loss: {loss:.2f}, Acc: {acc}")
    
    # We return the accuracy to implement the Callback
    return acc

In [ ]:
learning_rate = 1e-3

metric = MulticlassAccuracy(num_classes=10)
loss_fn = nn.CrossEntropyLoss()
model = LeNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 20
best_acc = 0
counter = 0
patience = 5
delta = 1e-4
for t in range(epochs):
    print(f"-------------------------------\nEpoch {t+1}\n-------------------------------")

    model.train()
    train(train_dataloader, model, metric, loss_fn, optimizer, t, device)
    
    model.eval()
    val_acc = test(val_dataloader, model, metric, loss_fn, t, device)

    # Callback: if the model does not improve significantly after some eopchs, stop training to prevent overfitting
    if (val_acc > best_acc and val_acc-best_acc >= delta):
        best_acc = val_acc
        counter = 0
    elif (val_acc > best_acc and val_acc-best_acc < delta):
        best_acc = val_acc
        counter += 1
    else:
        counter +=1

    if counter >= patience:
        print(f"No significant improvement after {patience} epochs, stopping early.")
        break

print("Done!")
writer.flush()

## Results

In [ ]:
#Plot the curve of the metrics during training
%tensorboard --logdir=runs

In [ ]:
# Evaluate the model on the test set
model.eval()
outputs_list = []
with torch.no_grad():
    for (X, y) in iter(test_dataloader):
        X = X.to(device).float()
        y = y.to(device).long()
        pred = model(X)
        outputs_list.append({"preds": pred, "labels": y})

preds = torch.cat([x["preds"] for x in outputs_list]).argmax(axis=1).detach().cpu()
labels = torch.cat([x["labels"] for x in outputs_list]).detach().cpu()

acc = metric(preds, labels)

print("Accuracy on test set: {0}".format(acc))

# II. Second step: Simple Convolutionnal Neural Network on more complex data.

### Collecting and loading the data

In [ ]:
# We need to create a custom dataset that extends the Dataset class
class MedMNISTDataset(Dataset):
    def __init__(self, label_array, img_array, transform=None, target_transform=None):

        # Load the npy arrays where our data is stored
        self.img_labels = np.load(label_array)
        self.img_array = np.load(img_array)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

        # Pair each image with its label and apply transformations
        image = self.img_array[idx]
        label = self.img_labels[idx, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# preprocessing
data_transform = Compose([
    ToTensor(),
])

# load the data
xTrain='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/train_images.npy'
xVal='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/val_images.npy'
xTest='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/test_images.npy'

yTrain='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/train_labels.npy'
yVal='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/val_labels.npy'
yTest='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/pathmnist/test_labels.npy'


# Wrap the data into datasets
train_dataset = MedMNISTDataset(yTrain,xTrain, data_transform)
val_dataset = MedMNISTDataset(yVal,xVal, data_transform)
test_dataset = MedMNISTDataset(yTest,xTest, data_transform)



batch_size = 64
# Wrap datasets into dataloaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


### Exploring the data

## Building and training the model

### Architecture

In [ ]:
class LeNetMed(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3,6,(5,5),padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d((2,2),stride=2),
            nn.Conv2d(6,16,(5,5)),
            nn.Sigmoid(),
            nn.AvgPool2d((2,2),stride=2),
        )
        self.dense_stack = nn.Sequential(
            nn.LazyLinear(120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,9),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.flatten(x)
        logits = self.dense_stack(x)

        return logits

model_med = LeNetMed()
print(model_med)

In [ ]:
batch = next(iter(train_dataloader))
yhat = model_med(batch[0])

make_dot(yhat, params=dict(list(model_med.named_parameters()))).render("model_med_torchviz", format="png")

### Training

In [ ]:
learning_rate = 1e-3

metric = MulticlassAccuracy(num_classes=9)
loss_fn = nn.CrossEntropyLoss()
model_med = LeNetMed().to(device)

optimizer = torch.optim.AdamW(model_med.parameters(), lr=learning_rate)
epochs = 5
best_acc = 0
counter = 0
patience = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_med, metric, loss_fn, optimizer, t, device)
    val_acc = test(val_dataloader, model_med, metric, loss_fn, t, device)
    if (val_acc > best_acc and val_acc-best_acc >= 1e-4):
        best_acc = val_acc
        counter = 0
    elif (val_acc > best_acc and val_acc-best_acc < 1e-4):
        best_acc = val_acc
        counter += 1
    else:
        counter +=1

    if counter >= patience:
        print(f"No significant improvement after {patience} epochs, stopping early.")
        break
print("Done!")
writer.flush()

## Results

In [ ]:
#Plot the curve of the metrics during training
%tensorboard --logdir=runs 

In [ ]:
model.eval()
outputs_list = []
with torch.no_grad():
    for (X, y) in iter(test_dataloader):
        X = X.to(device).float()
        y = y.to(device).long()
        pred = model(X)
        outputs_list.append({"preds": pred, "labels": y})

preds = torch.cat([x["preds"] for x in outputs_list]).argmax(axis=1).detach().cpu()
labels = torch.cat([x["labels"] for x in outputs_list]).detach().cpu()

acc = metric(preds, labels)

print("Accuracy on test set: {0}".format(acc))

# III. Third step: Transfer Learning

In [ ]:
from torchvision.models import vgg16, VGG16_Weights
weights=VGG16_Weights.DEFAULT
preprocess = weights.transforms()

from PIL import Image

import glob

### Collecting and loading the data

In [ ]:
# WARNING: do not run this cell now. It is very time consuming. Run the next one and come back here if you have time later.
datasetRoot='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/animal-10/raw-img/'
classes = ['mucca', 'elefante', 'gatto', 'cavallo', 'scoiattolo', 'ragno', 'pecora', 'farfalla', 'gallina', 'cane']
nbClasses = len(classes)

#training data

rootTrain = datasetRoot+'train/'
classLabel = 0

xTrain = np.empty(shape=(0,224,224,3))
yTrain = []
first = True
for cl in tqdm(classes):
    listImages = glob.glob(rootTrain+cl+'/*')
    yTrain += [classLabel]*100
    for pathImg in tqdm(listImages[:100]):
        img = Image.open(pathImg).resize((224, 224))
        im = preprocess(img)
        im = np.array(img)
        im = np.expand_dims(im, axis=0)
        xTrain = np.vstack([xTrain, im])
    classLabel += 1
print(len(yTrain))
print(xTrain.shape)


In [ ]:
# Load the training data

datasetRoot='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/animal-10/raw-img/'
classes = ['mucca', 'gatto']
nbClasses = len(classes)

rootTrain = datasetRoot+'train/'
classLabel = 0

xTrain = np.empty(shape=(0,224,224,3))
yTrain = []
first = True
for cl in tqdm(classes):
    listImages = glob.glob(rootTrain+cl+'/*')
    yTrain += [classLabel]*500
    for pathImg in tqdm(listImages[:500]):
        img = Image.open(pathImg).resize((224, 224))
        im = preprocess(img)
        im = np.array(img, dtype = "float32")
        im = np.expand_dims(im, axis=0)
        xTrain = np.vstack([xTrain, im])
    classLabel += 1
print(len(yTrain))
print(xTrain.shape)

In [ ]:
# Load the test data

datasetRoot='/content/drive/MyDrive/Colab Notebooks/Computer Vision/Lab3/animal-10/raw-img/'

rootTest = datasetRoot+'test/'
classLabel = 0

xTest = np.empty(shape=(0,224,224,3))
yTest = []

for cl in tqdm(classes):
    listImages = glob.glob(rootTest+cl+'/*')
    yTest += [classLabel]*len(listImages)
    for pathImg in tqdm(listImages):
        img = Image.open(pathImg).resize((224, 224))
        im = preprocess(img)
        im = np.array(img, dtype = "float32")
        im = np.expand_dims(im, axis=0)
        xTest = np.vstack([xTest, im])
    classLabel += 1
print(len(yTest))
print(xTest.shape)

In [ ]:
class AnimalDataset(Dataset):
    def __init__(self, label_array, img_array):
        self.img_labels = label_array
        self.img_array = img_array

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = torch.tensor(self.img_array[idx].transpose(2, 0, 1), dtype = torch.float)
        label = torch.tensor(self.img_labels[idx], dtype = torch.float)
        return image, label

In [ ]:
train_val_dataset = AnimalDataset(yTrain,xTrain)
test_dataset = AnimalDataset(yTest, xTest)

train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [0.8, 0.2])

batch_size = 64

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Exploring the data

In [ ]:
#Let's plot an image

plt.imshow((xTrain[100] * 255).astype(np.uint8))
print(yTrain[100])

## Pre-trained model

### Architecture

In [ ]:
# Load pre-trained VGG16 model
vgg16_model = vgg16(weights = VGG16_Weights.DEFAULT)

# Customize pooling and classifier layers of the model
vgg16_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

vgg16_model.classifier = nn.Sequential(
    nn.Linear(in_features=512, out_features=1024, bias=True),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=2, bias=True),
    nn.Softmax(),
  )

# Freeze all layers except for the classifier's
for child in list(vgg16_model.children())[:-1]:
    for param in list(child.parameters()):
        param.requires_grad = False

vgg16_classifier_model = vgg16_model

print(vgg16_classifier_model)

In [ ]:
summary(vgg16_classifier_model.to(device), (3,224,224))

In [ ]:
## Let us plot the model's graph and each layer's shapes
batch = list(train_dataloader)[0]
yhat = vgg16_classifier_model(batch[0].to(device))

make_dot(yhat, params=dict(list(vgg16_classifier_model.named_parameters()))).render("model_animal_torchviz", format="png")

### Training

In [ ]:
learning_rate = 1e-2

metric = MulticlassAccuracy(num_classes=2)
loss_fn = nn.CrossEntropyLoss()

model = vgg16_classifier_model.to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
epochs = 20
best_acc = 0
counter = 0
patience = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    train(train_dataloader, model, metric, loss_fn, optimizer, t, device)

    model.eval()
    val_acc = test(val_dataloader, model, metric, loss_fn, t, device)
    if (val_acc > best_acc and val_acc-best_acc >= 1e-4):
        best_acc = val_acc
        counter = 0
    elif (val_acc > best_acc and val_acc-best_acc < 1e-4):
        best_acc = val_acc
        counter += 1
    else:
        counter +=1

    if counter >= patience and best_acc >= val_acc:
        print(f"No significant improvement after {patience} epochs, stopping early.")
        break
print("Done!")
writer.flush()

## Fine-tune the network

In [ ]:
# Let us check all the named parameters in the model
i=0
for name, param in model.features.named_parameters():
    i+=1
    print(i,name)

In [ ]:
# Unfreeze all layers starting from the last convolutional layer

for param in list(vgg16_classifier_model.features.named_parameters())[-2:]:
    param[1].requires_grad = True

In [ ]:
# We use a smaller learning rate to fine-tune the model
learning_rate = 1e-4

metric = MulticlassAccuracy(num_classes=2)
loss_fn = nn.CrossEntropyLoss()
model = vgg16_classifier_model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
epochs = 20
best_acc = 0
counter = 0
patience = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    train(train_dataloader, model, metric, loss_fn, optimizer, t, device)

    model.eval()
    val_acc = test(test_dataloader, model, metric, loss_fn, t, device)
    if (val_acc > best_acc and val_acc-best_acc >= 1e-4):
        best_acc = val_acc
        counter = 0
    elif (val_acc > best_acc and val_acc-best_acc < 1e-4):
        best_acc = val_acc
        counter += 1
    else:
        counter +=1

    if counter >= patience and best_acc >= val_acc:
        print(f"No significant improvement after {patience} epochs, stopping early.")
        break
print("Done!")
writer.flush()

## Results

In [ ]:
# Plot the curves of the metrics
%tensorboard --logdir=runs

In [ ]:
# Let us compute the confusion matrix
outputs_list = []
with torch.no_grad():
    for (X, y) in list(test_dataloader):
        X = X.to(device).float()
        y = y.to(device).long()
        pred = model(X)
        test_loss = loss_fn(pred, y)
        # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        outputs_list.append({"preds": pred})

preds = torch.cat([x["preds"] for x in outputs_list]).detach().cpu()

y_classes = preds.argmax(axis=-1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true = yTest, y_pred = y_classes)
print(cm)

In [ ]:
# Nicer Display:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.show()